In [3]:
from datasets import load_dataset

ds = load_dataset("rubrix/research_papers_multi-label", split='train', streaming=True)

In [66]:
ds_head = ds.take(1000)

In [67]:
papers=list(ds_head)

In [8]:
df = pd.read_csv("hf://datasets/Devika03/Research_Paper_Summarization_Dataset/Research paper summarization dataset - Sheet1.csv")

In [9]:
papers = df.to_dict(orient='records')

In [21]:
df

,Paper Id,Image ID,Paper Title,Key Words,Abstract,Conclusion,Paper Type,Summarization,Topic
0,1.0,NaN,Multi-document Summarization via Deep Learning...,NaN,Multi-document summarization (MDS) is an effec...,"In this article, we have presented the first c...",Text summarization,This article presents a systematic overview of...,Natural Language Processing
1,2.0,NaN,NLP based Machine Learning Approaches for Text...,NaN,"Due to the plethora of data available today, t...",We have seen that due to abundant availability...,Natural Language Processing,The article discusses the importance of text s...,Natural Language Processing
2,3.0,NaN,Abstractive text summarization using LSTM-CNN ...,Text mining . Abstractive text summarization ....,"Abstractive Text Summarization (ATS), which i...","In this paper, we develop a novel LSTM-CNN bas...",Text summarization,The article presents a new framework for abstr...,Natural Language Processing
3,4.0,NaN,DEXPERTS: Decoding-Time Controlled Text Genera...,NaN,Despite recent advances in natural language\ng...,"We present DEXPERTS, a method for controlled\n...",Text generation,The paper proposes a method called DEXPERTS fo...,Natural Language Processing
4,5.0,NaN,A Survey of Knowledge-enhanced Text Generation,NaN,The goal of text-to-text generation is to make...,"In this survey, we present a comprehensive rev...",Text generation,The paper discusses the challenges in text-to-...,Natural Language Processing
...,...,...,...,...,...,...,...,...,...
329,326.0,NaN,Sentiment Analysis in Social Media and Its App...,Sentiment analysis; Big data; Social media,Twitter and sentiment analysis application can...,The conducted systematic literature review pro...,Deep Learning and Machine Learning,The article discusses how sentiment analysis i...,NaN
330,327.0,NaN,Expert Systems With Applications,Sentiment analysis \r\nSocial media \r\nTwitte...,Sentiment analysis has proven to be a valuable...,"Regarding technologies, it should be noted tha...",Deep Learning and Machine Learning,The article discusses the importance of sentim...,NaN
331,328.0,NaN,Sentiment Analysis for Social Media,"Sentiment Analysis, Natural Language Processin...","Sentiment analysis, the automated extraction o...",using the sentiment scores for sentiments rega...,Deep Learning and Machine Learning,Sentiment analysis is the automated extraction...,NaN
332,329.0,NaN,A review on sentiment analysis and emotion det...,Afective computing · Natural language processi...,Social networking platforms have become an ess...,"In this paper, a review of the existing techni...",Deep Learning and Machine Learning,Social networking platforms have become a popu...,NaN


In [60]:
papers[3]['inputs']['abstract']

'  The stochastic Landau--Lifshitz--Gilbert (LLG) equation coupled with the\nMaxwell equations (the so called stochastic MLLG system) describes the creation\nof domain walls and vortices (fundamental objects for the novel nanostructured\nmagnetic memories). We first reformulate the stochastic LLG equation into an\nequation with time-differentiable solutions. We then propose a convergent\n$\\theta$-linear scheme to approximate the solutions of the reformulated system.\nAs a consequence, we prove convergence of the approximate solutions, with no or\nminor conditions on time and space steps (depending on the value of $\\theta$).\nHence, we prove the existence of weak martingale solutions of the stochastic\nMLLG system. Numerical results are presented to show applicability of the\nmethod.\n'

In [13]:
from sentence_transformers import SentenceTransformer

In [14]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [68]:
from tqdm.auto import tqdm

for paper in tqdm(papers):
  title = paper['inputs']['title']
  abstract = paper['inputs']['abstract']
  paper['v_t'] = model.encode(title)
  paper['v_a'] = model.encode(abstract)
  paper['v_at'] = model.encode(title + ' ' + abstract)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [63]:
papers[0]

{'text': None,
 'inputs': {'abstract': "  Predictive models allow subject-specific inference when analyzing disease\nrelated alterations in neuroimaging data. Given a subject's data, inference can\nbe made at two levels: global, i.e. identifiying condition presence for the\nsubject, and local, i.e. detecting condition effect on each individual\nmeasurement extracted from the subject's data. While global inference is widely\nused, local inference, which can be used to form subject-specific effect maps,\nis rarely used because existing models often yield noisy detections composed of\ndispersed isolated islands. In this article, we propose a reconstruction\nmethod, named RSM, to improve subject-specific detections of predictive\nmodeling approaches and in particular, binary classifiers. RSM specifically\naims to reduce noise due to sampling error associated with using a finite\nsample of examples to train classifiers. The proposed method is a wrapper-type\nalgorithm that can be used with 

In [16]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

In [69]:
index_settings = {
    'settings':
    {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    'mappings':
    {
        'properties':
        {
            'inputs' : {
                'properties': {
                    'abstract' : {'type': 'text'},
                    'title' : {'type': 'text'},
                }
            },
            'v_t': {
                "type": "dense_vector", 
                "dims": 384, 
                "index": True, 
                "similarity": "cosine"
            },
            'v_a': {
                "type": "dense_vector", 
                "dims": 384, 
                "index": True, 
                "similarity": "cosine"
            },
            'v_at': {
                "type": "dense_vector", 
                "dims": 384, 
                "index": True, 
                "similarity": "cosine"
            }
        }
    }
}

index_name = 'academic_papers'

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'academic_papers'})

In [70]:
for paper in tqdm(papers):
    es_client.index(index=index_name, document=paper)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [100]:
v_q = model.encode('Find me a paper on the causes of Alzheimer\'s')

In [101]:
search_query = {
    "field": "v_at",
    "query_vector": v_q,
    "k": 5,
    "num_candidates":10000
}

es_results = es_client.search(
    index=index_name,
    knn=search_query, 
    source=['inputs']
)

In [102]:
es_results['hits']['hits']

[{'_index': 'academic_papers',
  '_id': 'zXeYYJEBbDRXV_TJ6gJS',
  '_score': 0.7078873,
  '_source': {'inputs': {'abstract': "  A sequence of pathological changes takes place in Alzheimer's disease, which\ncan be assessed in vivo using various brain imaging methods. Currently, there\nis no appropriate statistical model available that can easily integrate\nmultiple imaging modalities, being able to utilize the additional information\nprovided from the combined data. We applied Gaussian graphical models (GGMs)\nfor analyzing the conditional dependency networks of multimodal neuroimaging\ndata and assessed alterations of the network structure in mild cognitive\nimpairment (MCI) and Alzheimer's dementia (AD) compared to cognitively healthy\ncontrols.\nData from N=667 subjects were obtained from the Alzheimer's Disease\nNeuroimaging Initiative. Mean amyloid load (AV45-PET), glucose metabolism\n(FDG-PET), and gray matter volume (MRI) was calculated for each brain region.\nSeparate GGMs were e

In [30]:
from openai import OpenAI

client = OpenAI(
    base_url = 'https://api.groq.com/openai/v1',
    api_key = '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'
)

In [27]:
def elastic_search(query, index='academic_papers_v'):
    v_q = model.encode(query)
    
    search_query = {
        "field": "v_at",
        "query_vector": v_q,
        "k": 5,
        "num_candidates":10000
    }

    es_results = es_client.search(
        index=index,
        knn=search_query, 
        source=['inputs', 'pid']
    )

    return es_results['hits']['hits']

In [28]:
def prompt_builder(query, results):
    prompt_template =  """
    You are a librarian trying to help a researcher look for the research papers they are looking for.
    Make sure you only use the facts from the context to answer the question and nothing outside.

    Context: {context}

    Questions: {query}
    """
    context = ''
    for doc in results:
        context += f"\nTitle: {doc['_source']['inputs']['title']}\nAbstract: {doc['_source']['inputs']['abstract']}"

    prompt = prompt_template.format(query=query, context = context)

    return prompt

In [32]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gemma2-9b-it',
        messages = [{'role': 'user', 'content': prompt}]
    )

    return response.choices[0].message.content

In [11]:
def rag(query):
    result_docs = elastic_search(query)
    prompt = prompt_builder(query, result_docs)
    return llm(prompt)

In [26]:
elastic_search('I am look for a paper about alzheimers disease in younger people')

ObjectApiResponse({'took': 59, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5, 'relation': 'eq'}, 'max_score': 0.72312623, 'hits': [{'_index': 'academic_papers_v', '_id': 's5WeypEBp7yFllmOoyE3', '_score': 0.72312623, '_source': {'inputs': {'abstract': "  A sequence of pathological changes takes place in Alzheimer's disease, which\ncan be assessed in vivo using various brain imaging methods. Currently, there\nis no appropriate statistical model available that can easily integrate\nmultiple imaging modalities, being able to utilize the additional information\nprovided from the combined data. We applied Gaussian graphical models (GGMs)\nfor analyzing the conditional dependency networks of multimodal neuroimaging\ndata and assessed alterations of the network structure in mild cognitive\nimpairment (MCI) and Alzheimer's dementia (AD) compared to cognitively healthy\ncontrols.\nData from N=667 subjects were obtained from

In [34]:
elastic_search('Can you give me a paper regarding the Riemann Hypothesis?')

[{'_index': 'academic_papers_v',
  '_id': 'o5WeypEBp7yFllmOqCIY',
  '_score': 0.68918157,
  '_source': {'inputs': {'abstract': '  This paper deals with some simple results about spherical functions of type\n$\\delta$, namely new integral formulas, new results about behavior at infinity\nand some facts about the related $C_\\sigma$ functions.\n',
    'title': 'Spherical Functions on Riemannian Symmetric Spaces'},
   'pid': '26b4562d'}},
 {'_index': 'academic_papers_v',
  '_id': 'S5WeypEBp7yFllmOoSEV',
  '_score': 0.6731682,
  '_source': {'inputs': {'abstract': "  The purpose of this paper is to formulate and study a common refinement of a\nversion of Stark's conjecture and its $p$-adic analogue, in terms of Fontaine's\n$p$-adic period ring and $p$-adic Hodge theory. We construct period-ring-valued\nfunctions under a generalization of Yoshida's conjecture on the transcendental\nparts of CM-periods. Then we conjecture a reciprocity law on their special\nvalues concerning the absolute Frob